# Quickstart in 5 minutes

In order to run your first Deepchecks Suite all you need to have is the data and model that you wish to validate. More specifically, you need:

- Your train and test data (in Pandas DataFrames or Numpy Arrays)
- (optional) A [supported model](../../user-guide/supported_models.rst) (including XGBoost, scikit-learn models, and many more). Required for running checks that need the model's predictions for running.

To run your first suite on your data and model, you need only a few lines of code, that start here: [Define a Dataset Object](#Define-a-Dataset-Object).

In [13]:
# If you don't have deepchecks installed yet:
import sys
!{sys.executable} -m pip install deepchecks -U --quiet #--user

## Load Data, Split Train-Val, and Train a Simple Model

For the purpose of this guide we'll use the simple iris dataset and train a simple random forest model for multiclass classification:

In [14]:
# General imports
import pandas as pd
import numpy as np

from deepchecks.datasets.classification import iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load Data
iris_df = iris.load_data(data_format='Dataframe', as_train_test=False)
label_col = 'target'
df_train, df_test = train_test_split(iris_df, stratify=iris_df[label_col], random_state=0)

# Train Model
rf_clf = RandomForestClassifier()
rf_clf.fit(df_train.drop(label_col, axis=1), df_train[label_col]);

## Define a Dataset Object

Initialize the [Dataset object](../../user-guide/dataset_object.rst), stating the relevant metadata about the dataset (e.g. the name for the label column) <br>
Check out the Dataset's attributes to see which additional special columns can be declared and used (e.g. date column, index column).

In [15]:
from deepchecks import Dataset

# We explicitly state that this dataset has no categorical features, otherwise they will be automatically inferred
# If the dataset has categorical features, the best practice is to pass a list with their names

ds_train = Dataset(df_train, label=label_col, cat_features=[])
ds_test =  Dataset(df_test,  label=label_col, cat_features=[])

## Run a Deepchecks Suite

### Run the full suite

Use the `full_suite` that is a collection of (most of) the prebuilt checks. <br>
Check out the [when should you use deepchecks guide](../../user-guide/when_should_you_use.html) for some more info about the existing suites and when to use them.

In [16]:
from deepchecks.suites import full_suite

suite = full_suite()

In [17]:
suite.run(train_dataset=ds_train, test_dataset=ds_test, model=rf_clf)

Full Suite:   0%|          | 0/36 [00:00<?, ? Check/s]

### Run the integrity suite

If you still haven't started modeling and just have a single dataset, you can use the ``single_dataset_integrity``:

In [18]:
from deepchecks.suites import single_dataset_integrity

integ_suite = single_dataset_integrity()
integ_suite.run(ds_train)

Single Dataset Integrity Suite:   0%|          | 0/8 [00:00<?, ? Check/s]

## Run a Deepchecks Check

If you want to run a specific check, you can just import it and run it directly.

Check out the [Check Demonstrations](../examples/index.rst) in the examples or the [API Reference](../../api/index.rst) for more info about the existing checks and their parameters.

In [19]:
from deepchecks.checks import TrainTestLabelDrift

In [20]:
check = TrainTestLabelDrift()
result = check.run(ds_train, ds_test)
result

Train Test Label Drift Calculate label drift between train dataset and test dataset, using statistical measures. Read More... Additional Outputs 
 The Drift score is a measure for the difference between two distributions, in this check - the test
 and train distributions. The check shows the drift score and distributions for the label.

and also inspect the result value which has a check-dependant structure:

In [21]:
result.value

{'Drift score': 0.002507306267756066, 'Method': 'PSI'}

## Edit an Existing Suite

### Inspect suite and remove condition

We can see that the single feature contribution failed, both for test and for train. Since this is a very simple dataset with few features and this behavior is not necessarily problematic, we will remove the existing conditions for the PPS

In [22]:
# Lets first print the suite to find the conditions that we want to change:
suite

Full Suite: [
	0: ModelInfo
	1: ColumnsInfo
	2: ConfusionMatrixReport
	3: PerformanceReport
		Conditions:
			0: Train-Test scores relative degradation is not greater than 0.1
	4: RocReport(excluded_classes=[])
		Conditions:
			0: AUC score for all the classes is not less than 0.7
	5: SimpleModelComparison
		Conditions:
			0: Model performance gain over simple model is not less than 10%
	6: ModelErrorAnalysis
		Conditions:
			0: The performance difference of the detected segments must not be greater than 5%
	7: CalibrationScore
	8: TrustScoreComparison
		Conditions:
			0: Mean trust score decline is not greater than 20%
	9: RegressionSystematicError
		Conditions:
			0: Bias ratio is not greater than 0.01
	10: RegressionErrorDistribution
		Conditions:
			0: Kurtosis value is not less than -0.1
	11: BoostingOverfit
		Conditions:
			0: Test score over iterations doesn't decline by more than 5% from the best score
	12: UnusedFeatures
		Conditions:
			0: Number of high variance unused featur

In [23]:
# now we can use the check's index and the condition's number to remove it:
print(suite[6])
suite[6].remove_condition(0)

ModelErrorAnalysis
	Conditions:
		0: The performance difference of the detected segments must not be greater than 5%


In [24]:
# print and see that the condition was removed
suite[6]

ModelErrorAnalysis

If we now re-run the suite, all of the existing conditions will pass.

*Note: the check we manipulated will still run as part of the Suite, however it won't appear in the Conditions Summary since it no longer has any conditions defined on it. You can still see its display results in the Additional Outputs section*

**For more info about working with conditions, see the detailed configuring conditions guide.**